# STM Science Assessor
This notebook aims to ingest a table of source/observation specs with their achievability rating per detector model (as generated by prior notebook, the Observation Targets Assessor), and convolve with the Science STM objectives, each of which involves one or more of those source/observation specs. The output should be a similar table to the input, but with the source/observvation specs replaced by objectives.

To run this on Google Colab:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnGBaker/GWI-metrics/blob/main/Notebooks/STM_Science_Assessor.ipynb)

### Some basic setup

In [ ]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/JohnGBaker/GWI-metrics.git
    src='GWI-metrics/src/'
else:
    src='../src/'
!mkdir -p '../plots/'

# Import
import os
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append(src)
import constants
import metrics
import sources
import concepts
import subsystems
import background
import pandas as pd
import dataframe_image as dfi
from glob import glob
import re
from astropy.coordinates import Angle
from astropy.cosmology import WMAP9 as cosmo
from astropy import units as u

LARGENUM = 1.0e10 # This is a hard-coded number to beat
verbose = False

### Define the set of concepts
Here we load in a set of the pre-defined concepts, or you can define your own

In [ ]:
missionNames = (
    'LISACBE',
    'LISASciRDv1',
    'TwinLISA',
    'LISAGrande',
    'LISAU',
    'GoBIGLISA',
    'ALIA',
    'ALIAlowL')
missions=[concepts.menu[mission] for mission in missionNames]

Nmissions = len(missionNames)

for mission in missions:
    mission=background.add2model(mission)
#model = concepts.LISACBE.copy()
#modelName = model.get('label')
#model = background.add2model(model)     # add galactic background model

### Load the observation target data
Load information about the target sources and the observation quality targets from the STM.
The data are from https://nasa.sharepoint.com/:x:/t/GravitationalWaveImager/EaaeMC7L-2NJpFCQloYbSnoBDZag_cvFWR5_BjRoFAD6Tw?e=uWr4mU

In [ ]:
# Read in the most recent spec file produced by other Notebook:
datadir='../Data/'
files=glob(datadir+'ScoredObservations_net_*.csv')
files.sort(key=os.path.getmtime)
specfile=files[-1]
print('Most recent specs file is:',specfile)

# Are objective names actually stored?
# A: not originally -- had to re-run earlier notbook with the "index=None" removed from the CVS save command

speclist = []
with open(specfile) as f:
    for row in f:
        speclist.append(row.split(",")[0])

speclist.pop(0) # remove the top item, which is empty "corner" element of table
print("specs are: ",speclist)

Nspecs = len(speclist)

#specarray = np.loadtxt(specfile,delimiter=',',skiprows=1,usecols = range(1,Nmissions+1))

rawspecarray = np.genfromtxt(specfile,delimiter=',')

specarray=rawspecarray[1:Nspecs+1,1:Nmissions+1]
#print("first row of specarray is ",specarray[0,])



    

In [ ]:
STMfiles=glob('../Data/ScienceSTM*.csv')
STMfiles.sort(key=os.path.getmtime)
file=STMfiles[-1]
#file=glob('../Data/ScienceSTM*.csv')[0]
print('Science STM file is:',file)

#Read file
df=pd.read_csv(file,header=1)
print(len(df),'rows read from file.')

if True:
    #Drop empty rows
    for i,row in df.iterrows():
        if row.isnull().all():
            df=df.drop(i)
    #Fill empties in first column with values above

    nrows = len(df. index) 

    newdata = np.zeros((nrows,Nmissions))

    val=float('nan')
    
    col='Astrophysical Parameters (Level 1 Measurement Req)'
    requirementNames=df[col].values
    if verbose:
        print("requirement Names are ",requirementNames)

    analysisVal=float('nan')
    analysisCol='Specific Analyses'
    analysisNames=df[analysisCol].values

    if verbose:
        print("analyses are: ",analysisNames)
    
    analysisCodes = analysisNames
    for j in range(len(analysisNames)):
        analysisName_here = analysisNames[j]
        #print("this analysis names is ",analysisName_here)
        #found = re.search("\w\w-\w\w",analysisName_here).group(1)
        found = re.findall("\w\w-\w\w",analysisName_here)
        #print("analysis Name ",analysisName_here,' has code ',found[0])
        analysisCodes[j] = found[0]
        
    actionVal=float('nan')
    actionCol='Science Actions'
    actionNames=df[actionCol].values

    cleanedList = [x for x in actionNames if str(x) != 'nan']
    actionNames = cleanedList
    actionCodes = actionNames
    for j in range(len(actionNames)):
        actionName_here = actionNames[j]
        found = re.findall("\w\w-\w",actionName_here)
        actionCodes[j] = found[0]
    if verbose:
        print("actions are: ",actionNames)        
        print('action codes are: ',actionCodes)
    
    useAnalysis = np.full(len(analysisCodes),False)
    
    for i,row in df.iterrows(): #each row should be a specific analysis in analysisNames
        if row.isnull()[col]:
            val = ''
            #df.loc[i,col]=val
        else:
            val=df.loc[i,col]

        if row.isnull()[analysisCol]:
            df.loc[i,analysisCol]=analysisVal
        else:
            analysisVal=df.loc[i,analysisCol]    
            
        bestspecres_here = LARGENUM*np.ones(Nmissions)

        # parse to get values in form of 2.3.b == <single digit> + dot + <single digit> + dot + <single letter>
        # or perhaps just a comma-separated list that can be compared with other file
        speccases_here = val.split(",")

        Nspeccases_here = len(speccases_here)
        Nbadspecs = 0 # counting bad/missing specs for this analysis
        for ispeccases_here in range(Nspeccases_here): # loop over all measure requirements associated with this Specific Analysis
            speccase_here = (speccases_here[ispeccases_here]).strip() # remove leading whitespace
            # look for match with list of specs read in earlier
            # if it's present, combine with existing data for this requirement -- how?
            # for now: try smallest number
            specres_here = 2.0*LARGENUM*np.ones(Nmissions)
            try:
                specidx = speclist.index(speccase_here)
                specres_here = specarray[specidx,:]
            except ValueError:
                print("specification ",speccase_here," not generated by earlier Notebook! Ignoring")
                Nbadspecs = Nbadspecs + 1
           
            #bestspecres_here = np.minimum(bestspecres_here,specres_here)
            bestspecres_here = np.fmin(bestspecres_here,specres_here) # get minimum over all non-NaN values

        #print("For this requirement, the best spec results for each detector are ",bestspecres_here)
        if Nbadspecs<Nspeccases_here:
            newdata[i,:] = bestspecres_here
            useAnalysis[i] = True
            
        # This is a Ndetectors-length array for this particular Specific Analysis
        
    # Want to cut the main analysis list down using the useAnalysis array
    goodAnalysisCodes=analysisCodes[useAnalysis]
    goodAnalysisData=newdata[useAnalysis,:]


### Perform the observations and assessment
We first compute the SNR and then the angular resolution

In [ ]:
#adfnew = pd.DataFrame(newdata,index=analysisCodes,columns=missionNames)
adfnew = pd.DataFrame(goodAnalysisData,index=goodAnalysisCodes,columns=missionNames)
print(adfnew)

#adf=adf.dropna()
def shrink():
    return [dict(selector="th",
                 props=[("font-size", "6pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")])
]
def zoom():
    return [
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]


s=adfnew.style.background_gradient(vmin=0.1,vmax=10,cmap='RdYlGn_r').set_properties(**{'max-width': '120px', 'font-size': '6pt'})
#s=s.set_table_styles(shrink()+zoom())
s=s.set_caption("Net performance")
display(s)

saveSpecificAnalysesTable=True
if saveSpecificAnalysesTable:
    outimagename = datadir+'ScoredRequirements_net_'+dates+'.png'
    dfi.export(
        s,
        outimagename
    )

if True:
    dates = date.today().strftime("%Y_%m_%d")
    outfile=datadir+'ScoredRequirements_net_'+dates+'.csv'
    #adfnew.to_csv(outfile,index=False)
    adfnew.to_csv(outfile,index=True)
    print('Wrote to file:',outfile)


### Show assessments

### Make some plots

#### Plot sensitivity curves and waveform
Here we scale the waveform by delta-f, which is probably not quite right